# 🏆 K-League Pass Prediction - v9.2 Score Diff & Feature Clean

## 변경사항 (v9.1 → v9.2)
- **LSTM 입력에 점수차 추가**: 실시간 점수차 피처 (9D → 10D)
- **불필요 피처 제거**:
  - `total_distance` 제거 (mean_distance와 중복성)
  - `is_weekend` 제거 (영향 미미)

## LSTM 입력 (10D)
```
[dx, dy, dist, angle, time, dist_to_goal, angle_to_goal, dist_to_own_goal, dist_to_center, score_diff]
```

## 점수차 계산 방식
- 에피소드 내 Goal 이벤트 카운트
- 우리팀 골 - 상대팀 골
- 정규화: / 5.0 (큰 점수차 드묾)


#Part1 사전준비

In [ ]:
!pip install pandas numpy scikit-learn tqdm torch lightgbm -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tqdm import tqdm
import random
import os
import warnings
warnings.filterwarnings('ignore')

import torch
from torch import nn
from torch.nn import functional as F
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.utils.data import Dataset, DataLoader
import lightgbm as lgb

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")
if DEVICE == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Device: cuda
GPU: NVIDIA L4


## 02. 하이퍼파라미터

In [ ]:
# Seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# 경로
BASE_DIR = "/content/drive/MyDrive/ColabNotebooks/K_league/data_open_track1"
TRAIN_PATH = f"{BASE_DIR}/train.csv"
TEST_PATH = f"{BASE_DIR}/test.csv"
MATCH_INFO_PATH = f"{BASE_DIR}/match_info.csv"

# LSTM 설정
LSTM_HIDDEN_DIM = 128
LSTM_NUM_LAYERS = 2
LSTM_DROPOUT = 0.3
EMBEDDING_DIM = 16
BATCH_SIZE = 64
LSTM_EPOCHS = 100
LR = 0.001

# LGBM 설정
LGBM_PARAMS = {
    'objective': 'regression',
    'metric': 'rmse',
    'max_depth': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 127,
    'learning_rate': 0.03,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 20,
    'verbose': -1,
    'n_estimators': 2000,
    'random_state': SEED
}

print("✅ 설정 완료")

✅ 설정 완료


---
## 03. 데이터 로드 및 전처리

In [ ]:
print("📂 데이터 로드...")
train_df = pd.read_csv(TRAIN_PATH)
match_info = pd.read_csv(MATCH_INFO_PATH)

# 정렬
train_df = train_df.sort_values(['game_episode', 'time_seconds']).reset_index(drop=True)

print(f"✅ Train: {len(train_df):,}행")
print(f"✅ Match Info: {len(match_info)}경기")

📂 데이터 로드...
✅ Train: 356,721행
✅ Match Info: 228경기


### 3.1 Y축 Augmentation

In [ ]:
def augment_y_flip_sequences(sequences):
    """
    시퀀스 레벨에서 Y축 대칭 Augmentation
    Train 데이터에만 적용 (Val 누수 방지)
    """
    augmented = []
    for s in sequences:
        aug = {
            'continuous': s['continuous'].copy(),
            'types': s['types'].copy(),
            'results': s['results'].copy(),
            'target': s['target'].copy(),
            'last_abs_coords': s['last_abs_coords'].copy(),
            'episode_id': str(s['episode_id']) + '_aug',
            'meta_features': s['meta_features'].copy()
        }
        # Y축 반전
        aug['continuous'][:, 1] *= -1  # dy 반전
        aug['continuous'][:, 3] *= -1  # angle 반전
        aug['target'][1] *= -1  # target dy 반전
        aug['last_abs_coords'][1] = 1.0 - aug['last_abs_coords'][1]
        aug['meta_features']['last_y'] = 1.0 - aug['meta_features']['last_y']
        augmented.append(aug)

    return sequences + augmented

print("✅ Y축 대칭 증강 함수 정의 완료 (Train/Val 분할 후 적용)")

✅ Y축 대칭 증강 함수 정의 완료 (Train/Val 분할 후 적용)


### 3.2 Global 피처 준비 (Player, Match Context)

In [ ]:
# Type/Result 매핑
type_map = {name: i for i, name in enumerate(train_df['type_name'].unique())}
result_map = {name: i for i, name in enumerate(train_df['result_name'].fillna("Unknown").unique())}
NUM_TYPES, NUM_RESULTS = len(type_map), len(result_map)

print(f"Types: {NUM_TYPES}, Results: {NUM_RESULTS}")

# Player Role 추정 (KMeans)
def create_player_roles(df):
    stats = df.groupby('player_id').agg({
        'start_x': 'mean',
        'type_name': lambda x: list(x)
    }).reset_index()

    stats['is_gk'] = stats['type_name'].apply(
        lambda x: any(t in ['Catch', 'Parry', 'Goal Keeping'] for t in x)
    )
    stats['role'] = 0  # GK

    field = stats[~stats['is_gk']].copy()
    if len(field) > 3:
        km = KMeans(n_clusters=3, random_state=SEED, n_init=10).fit(field[['start_x']])
        idx = np.argsort(km.cluster_centers_.flatten())
        cmap = {old: new + 1 for new, old in enumerate(idx)}  # 1=DF, 2=MF, 3=FW
        field['role'] = pd.Series(km.labels_, index=field.index).map(cmap)
        stats.loc[field.index, 'role'] = field['role']

    return dict(zip(stats['player_id'], stats['role']))

role_map = create_player_roles(train_df)
print(f"✅ Player Roles: {len(role_map)}명")

# Player 통계
orig_train = train_df.copy()
orig_train['step_dx'] = orig_train['end_x'] - orig_train['start_x']
orig_train['step_dy'] = orig_train['end_y'] - orig_train['start_y']
orig_train['step_dist'] = np.sqrt(orig_train['step_dx']**2 + orig_train['step_dy']**2)

player_stats = orig_train.groupby('player_id').agg({
    'step_dist': 'mean',
    'step_dx': 'mean'
}).reset_index()
player_stats.columns = ['player_id', 'player_avg_dist', 'player_avg_dx']
player_stats_map = player_stats.set_index('player_id').to_dict('index')

print(f"✅ Player Stats: {len(player_stats_map)}명")

# ========== 팀 통계 추가 (NEW) ==========
# 팀별 평균 패스 거리
team_pass_stats = orig_train.groupby('team_id').agg({
    'step_dist': 'mean'
}).reset_index()
team_pass_stats.columns = ['team_id', 'team_avg_pass_dist']

# 팀별 점유율 (이벤트 수 비율로 근사)
total_events = len(orig_train)
team_possession = orig_train.groupby('team_id').size() / total_events
team_possession = team_possession.reset_index()
team_possession.columns = ['team_id', 'team_possession_ratio']

# 합치기
team_stats_df = team_pass_stats.merge(team_possession, on='team_id')
team_stats_map = team_stats_df.set_index('team_id').to_dict('index')

print(f"✅ Team Stats: {len(team_stats_map)}팀")
print(f"   평균 패스 거리: {team_stats_df['team_avg_pass_dist'].mean():.2f}m")
print(f"   점유율 범위: {team_stats_df['team_possession_ratio'].min():.3f} ~ {team_stats_df['team_possession_ratio'].max():.3f}")

Types: 26, Results: 9
✅ Player Roles: 446명
✅ Player Stats: 446명
✅ Team Stats: 12팀
   평균 패스 거리: 12.55m
   점유율 범위: 0.075 ~ 0.092


In [ ]:
# Match Context 처리
def process_match_info(matches):
    df = matches.copy()
    df['match_date_kst'] = pd.to_datetime(df['game_date']) + pd.Timedelta(hours=9)
    df['match_hour'] = df['match_date_kst'].dt.hour
    df['is_weekend'] = (df['match_date_kst'].dt.weekday >= 5).astype(int)

    # Rest days 계산
    home = df[['match_date_kst', 'home_team_id']].rename(columns={'home_team_id': 'team_id'})
    away = df[['match_date_kst', 'away_team_id']].rename(columns={'away_team_id': 'team_id'})
    full_schedule = pd.concat([home, away])
    full_schedule['date_only'] = full_schedule['match_date_kst'].dt.normalize()
    full_schedule = full_schedule.drop_duplicates().sort_values(['team_id', 'date_only'])
    full_schedule['prev_date'] = full_schedule.groupby('team_id')['date_only'].shift(1)
    full_schedule['rest_days'] = (full_schedule['date_only'] - full_schedule['prev_date']).dt.days
    full_schedule['rest_days'] = full_schedule['rest_days'].fillna(7).clip(0, 14)

    rest_map = dict(zip(
        zip(full_schedule['team_id'], full_schedule['date_only'].dt.date.astype(str)),
        full_schedule['rest_days']
    ))
    df['date_str'] = df['match_date_kst'].dt.date.astype(str)
    df['home_rest'] = df.apply(lambda x: rest_map.get((x['home_team_id'], x['date_str']), 7), axis=1)
    df['away_rest'] = df.apply(lambda x: rest_map.get((x['away_team_id'], x['date_str']), 7), axis=1)
    df['rest_diff'] = df['home_rest'] - df['away_rest']
    df['time_slot'] = pd.cut(df['match_hour'], bins=[-1, 17, 24], labels=[0, 1]).astype(int)

    return df

match_info_processed = process_match_info(match_info)
match_context = match_info_processed[[
    'game_id', 'match_hour', 'time_slot', 'is_weekend',
    'home_rest', 'away_rest', 'rest_diff',
    'home_team_id', 'away_team_id', 'home_score', 'away_score'
]].copy()

# 팀 평균 득점
home_gm = match_info.groupby('home_team_id')['home_score'].mean().to_dict()
away_gm = match_info.groupby('away_team_id')['away_score'].mean().to_dict()

print(f"✅ Match Context 준비 완료")

✅ Match Context 준비 완료


---
## 04. 피처 정의 및 계산

LSTM , LGBM

### 핵심: 흐름 vs 상황 분리

In [ ]:
"""
피처 분류 기준:

🔹 LSTM용 (시퀀스/흐름) - 매 타임스텝 변화가 중요
   - dx, dy (이동 벡터)
   - distance, angle (이동 크기/방향)
   - time delta
   - dist_to_goal, angle_to_goal, dist_to_own_goal (골대 관련)
   - dist_to_center (중앙선 거리)
   - score_diff (실시간 점수차) ← NEW

🔸 LGBM용 (상황 요약) - 현재 상태의 스냅샷
   - 위치: dist_to_goal, angle_to_goal, zone_id
   - 중앙선: dist_to_center, is_in_own_half
   - 통계: mean_dist, forward_ratio, seq_length
   - 컨텍스트: role, player_stats, team_stats, match_phase
   - Lag1: 직전 이동 정보
"""

# LSTM 시퀀스 피처 (매 스텝) - 점수차 추가
LSTM_CONTINUOUS_DIM = 10  # [dx, dy, dist, angle, time, dist_to_goal, angle_to_goal, dist_to_own_goal, dist_to_center, score_diff]

# LGBM 메타 피처 목록 (total_distance, is_weekend 제거)
LGBM_FEATURES = [
    # 위치 상태
    'last_x', 'last_y',

    # Zone
    'zone_id',

    # 골대 관련
    'dist_to_goal', 'angle_to_goal', 'goal_open_angle',
    'dist_to_own_goal',

    # 필드 영역
    'is_left_side', 'is_center', 'is_right_side',
    'is_final_third', 'is_near_touchline', 'min_dist_to_touchline',

    # 시퀀스 통계 요약 (total_distance 제거)
    'seq_length',
    'mean_distance', 'std_distance',
    'max_distance', 'min_distance',
    'mean_angle', 'std_angle',
    'forward_ratio', 'backward_ratio',
    'net_x_movement', 'net_y_movement',

    # 최근 이동
    'recent_dx_mean', 'recent_dy_mean', 'recent_dist_mean',

    # Lag1 피처
    'lag1_dx', 'lag1_dy', 'lag1_dist', 'lag1_angle',

    # 선수/팀 컨텍스트
    'role',
    'player_avg_dist', 'player_avg_dx',
    'team_avg_pass_dist', 'team_possession_ratio',
    'is_home',
    'is_set_piece',

    # 매치 컨텍스트 (is_weekend 제거)
    'match_phase',
    'time_delta', 'episode_time',
    'match_hour', 'time_slot',
    'current_team_rest', 'opp_team_rest', 'rest_diff',

    # 경기 상황
    'cumulative_score_diff', 'is_draw',
]

print(f"✅ LSTM: {LSTM_CONTINUOUS_DIM}D continuous (점수차 추가)")
print(f"✅ LGBM: {len(LGBM_FEATURES)} meta features (total_distance, is_weekend 제거)")

✅ LSTM: 10D continuous (점수차 추가)
✅ LGBM: 48 meta features (total_distance, is_weekend 제거)


In [ ]:
def convert_to_relative(episode_data, score_diffs):
    """
    절대좌표 → 상대좌표 + 골대/중앙선/점수차 피처 (LSTM용)

    Input:
        episode_data: shape (T, 3) = [x, y, time]
        score_diffs: shape (T,) = 각 시점의 점수차
    Output: shape (T, 10) = [dx, dy, dist, angle, time, dist_to_goal, angle_to_goal, dist_to_own_goal, dist_to_center, score_diff]
    """
    T = len(episode_data)
    result = np.zeros((T, 10), dtype=np.float32)

    GOAL_X, GOAL_Y = 1.0, 0.5
    OWN_GOAL_X, OWN_GOAL_Y = 0.0, 0.5
    CENTER_X = 0.5

    for i in range(T):
        x, y, t = episode_data[i]
        result[i, 5] = np.sqrt((x - GOAL_X)**2 + (y - GOAL_Y)**2)
        result[i, 6] = np.arctan2(GOAL_Y - y, GOAL_X - x) / np.pi
        result[i, 7] = np.sqrt((x - OWN_GOAL_X)**2 + (y - OWN_GOAL_Y)**2)
        result[i, 8] = np.abs(x - CENTER_X)
        result[i, 9] = score_diffs[i] / 5.0  # 점수차 정규화 (보통 -5 ~ +5)
        result[i, 4] = t

    result[0, :2] = episode_data[0, :2]

    for i in range(1, T):
        dx = episode_data[i, 0] - episode_data[i-1, 0]
        dy = episode_data[i, 1] - episode_data[i-1, 1]
        result[i, 0] = dx
        result[i, 1] = dy
        result[i, 2] = np.sqrt(dx**2 + dy**2)
        result[i, 3] = np.arctan2(dy, dx) / np.pi

    return result


def compute_zone_id(x, y):
    """필드를 3x3 Zone으로 분할 (0-8)"""
    x_zone = 0 if x < 1/3 else (1 if x < 2/3 else 2)
    y_zone = 0 if y < 1/3 else (1 if y < 2/3 else 2)
    return x_zone * 3 + y_zone


def compute_lgbm_features(ex, ey, sx, sy, norm_times, curr_types, curr_results,
                          last_x, last_y, type_map,
                          role, player_avg_dist, player_avg_dx,
                          team_avg_pass_dist, team_possession_ratio,
                          is_home, match_phase,
                          match_hour, time_slot,
                          current_team_rest, opp_team_rest, rest_diff,
                          cumulative_score_diff):
    """LGBM용 메타 피처 계산 (total_distance, is_weekend 제거)"""
    features = {}

    features['last_x'] = last_x
    features['last_y'] = last_y
    features['zone_id'] = compute_zone_id(last_x, last_y)

    # 골대 관련
    features['dist_to_goal'] = np.sqrt((last_x - 1.0)**2 + (last_y - 0.5)**2)
    features['angle_to_goal'] = np.arctan2(0.5 - last_y, 1.0 - last_x)
    features['dist_to_own_goal'] = np.sqrt(last_x**2 + (last_y - 0.5)**2)

    Y_NEAR, Y_FAR = 30.34/68, 37.66/68
    angle_near = np.arctan2(Y_NEAR - last_y, 1.0 - last_x)
    angle_far = np.arctan2(Y_FAR - last_y, 1.0 - last_x)
    features['goal_open_angle'] = np.abs(angle_far - angle_near)

    # 필드 영역
    features['is_left_side'] = int(last_y < 0.33)
    features['is_center'] = int(0.33 <= last_y <= 0.66)
    features['is_right_side'] = int(last_y > 0.66)
    features['is_final_third'] = int(last_x > 70/105)
    features['min_dist_to_touchline'] = min(last_y, 1.0 - last_y)
    features['is_near_touchline'] = int(features['min_dist_to_touchline'] < 5/68)

    # 시퀀스 통계 (total_distance 제거)
    features['seq_length'] = len(ex)

    if len(ex) > 1:
        dx = np.diff(ex)
        dy = np.diff(ey)
        distances = np.sqrt(dx**2 + dy**2)
        angles = np.arctan2(dy, dx)

        # total_distance 제거됨
        features['mean_distance'] = np.mean(distances)
        features['std_distance'] = np.std(distances) if len(distances) > 1 else 0
        features['max_distance'] = np.max(distances)
        features['min_distance'] = np.min(distances)
        features['mean_angle'] = np.mean(angles)
        features['std_angle'] = np.std(angles) if len(angles) > 1 else 0
        features['forward_ratio'] = np.mean(dx > 0)
        features['backward_ratio'] = np.mean(dx < 0)
        features['net_x_movement'] = ex[-1] - ex[0]
        features['net_y_movement'] = ey[-1] - ey[0]

        recent_n = min(3, len(dx))
        features['recent_dx_mean'] = np.mean(dx[-recent_n:])
        features['recent_dy_mean'] = np.mean(dy[-recent_n:])
        features['recent_dist_mean'] = np.mean(distances[-recent_n:])

        features['lag1_dx'] = dx[-1]
        features['lag1_dy'] = dy[-1]
        features['lag1_dist'] = distances[-1]
        features['lag1_angle'] = angles[-1]
    else:
        for k in ['mean_distance', 'std_distance', 'max_distance',
                  'min_distance', 'mean_angle', 'std_angle', 'forward_ratio',
                  'backward_ratio', 'net_x_movement', 'net_y_movement',
                  'recent_dx_mean', 'recent_dy_mean', 'recent_dist_mean',
                  'lag1_dx', 'lag1_dy', 'lag1_dist', 'lag1_angle']:
            features[k] = 0

    # 선수/팀 컨텍스트
    features['role'] = role
    features['player_avg_dist'] = player_avg_dist
    features['player_avg_dx'] = player_avg_dx
    features['team_avg_pass_dist'] = team_avg_pass_dist
    features['team_possession_ratio'] = team_possession_ratio
    features['is_home'] = is_home

    sp_types = [type_map.get(k, -1) for k in ['Corner', 'Freekick', 'Penalty', 'Kick Off'] if k in type_map]
    features['is_set_piece'] = int(any(t in sp_types for t in curr_types))

    # 매치 컨텍스트 (is_weekend 제거)
    features['match_phase'] = match_phase
    features['time_delta'] = norm_times[-1] - norm_times[-2] if len(norm_times) > 1 else 0
    features['episode_time'] = norm_times[-1] - norm_times[0] if len(norm_times) > 1 else 0
    features['match_hour'] = match_hour
    features['time_slot'] = time_slot
    features['current_team_rest'] = current_team_rest
    features['opp_team_rest'] = opp_team_rest
    features['rest_diff'] = rest_diff

    features['cumulative_score_diff'] = cumulative_score_diff
    features['is_draw'] = int(cumulative_score_diff == 0)

    return features

print("✅ 피처 계산 함수 정의 완료 (10D LSTM + 점수차)")

✅ 피처 계산 함수 정의 완료 (10D LSTM + 점수차)


---
## 06. 시퀀스 빌더

In [ ]:
def calculate_score_diff(episode_df, team_id, type_map):
    """
    에피소드 내 실시간 점수차 계산
    Goal 이벤트를 추적하여 누적 점수차 반환
    """
    score_diffs = []
    our_score = 0
    opp_score = 0

    goal_type_id = type_map.get('Goal', -1)

    for _, row in episode_df.iterrows():
        # Goal 이벤트 확인
        if row['type_name'] == 'Goal':
            if row['team_id'] == team_id:
                our_score += 1
            else:
                opp_score += 1

        score_diffs.append(our_score - opp_score)

    return np.array(score_diffs)


def build_train_sequences(df, type_map, result_map, match_context, role_map, player_stats_map, team_stats_map):
    """
    Train 에피소드별 시퀀스 구축 (v9.2: 점수차 추가)
    """
    sequences = []
    mc_dict = match_context.set_index('game_id').to_dict('index')
    grouped = df.groupby('game_episode')

    for episode_id, g in tqdm(grouped, desc="Train 시퀀스 구축"):
        g = g.sort_values('time_seconds').reset_index(drop=True)

        if len(g) < 2:
            continue

        # 정규화 좌표
        sx = g['start_x'].values / 105.0
        sy = g['start_y'].values / 68.0
        ex = g['end_x'].values / 105.0
        ey = g['end_y'].values / 68.0

        max_time = g['time_seconds'].max()
        norm_times = g['time_seconds'].values / max_time if max_time > 0 else np.zeros(len(g))

        curr_types = g['type_name'].map(type_map).fillna(0).values.astype(int)
        curr_results = g['result_name'].fillna("Unknown").map(result_map).fillna(0).values.astype(int)

        # 팀 정보
        team_id = g['team_id'].iloc[-1]

        # 점수차 계산 (NEW)
        score_diffs = calculate_score_diff(g, team_id, type_map)

        # Target: 마지막 end 좌표
        target_x, target_y = ex[-1], ey[-1]

        # Input: 마지막 이벤트 제외
        input_ex, input_ey = ex[:-1], ey[:-1]
        input_sx, input_sy = sx[:-1], sy[:-1]
        input_times = norm_times[:-1]
        input_types = curr_types[:-1]
        input_results = curr_results[:-1]
        input_score_diffs = score_diffs[:-1]  # 점수차도 마지막 제외

        if len(input_ex) < 1:
            continue

        last_x, last_y = input_ex[-1], input_ey[-1]

        # ========== LSTM용 시퀀스 (점수차 포함) ==========
        abs_coords = np.column_stack([input_ex, input_ey, input_times])
        continuous = convert_to_relative(abs_coords, input_score_diffs)

        # ========== LGBM용 메타 피처 ==========
        game_id = g['game_id'].iloc[0]
        mc = mc_dict.get(game_id, {})

        player_id = g['player_id'].iloc[-2] if len(g) > 1 else g['player_id'].iloc[0]
        role = role_map.get(player_id, 0)
        p_stats = player_stats_map.get(player_id, {'player_avg_dist': 0, 'player_avg_dx': 0})
        t_stats = team_stats_map.get(team_id, {'team_avg_pass_dist': 0, 'team_possession_ratio': 0})

        is_home = int(team_id == mc.get('home_team_id', -1))
        match_phase = int(g['time_seconds'].iloc[-1] // 900)
        cumulative_score_diff = input_score_diffs[-1] if len(input_score_diffs) > 0 else 0

        meta = compute_lgbm_features(
            input_ex, input_ey, input_sx, input_sy,
            input_times, input_types, input_results,
            last_x, last_y, type_map,
            role=role,
            player_avg_dist=p_stats.get('player_avg_dist', 0),
            player_avg_dx=p_stats.get('player_avg_dx', 0),
            team_avg_pass_dist=t_stats.get('team_avg_pass_dist', 0),
            team_possession_ratio=t_stats.get('team_possession_ratio', 0),
            is_home=is_home,
            match_phase=match_phase,
            match_hour=mc.get('match_hour', 19),
            time_slot=mc.get('time_slot', 1),
            current_team_rest=mc.get('home_rest', 7) if is_home else mc.get('away_rest', 7),
            opp_team_rest=mc.get('away_rest', 7) if is_home else mc.get('home_rest', 7),
            rest_diff=mc.get('rest_diff', 0) if is_home else -mc.get('rest_diff', 0),
            cumulative_score_diff=cumulative_score_diff
        )

        # Target: 상대좌표
        target_dx = target_x - last_x
        target_dy = target_y - last_y

        sequences.append({
            'continuous': continuous.astype(np.float32),
            'types': input_types,
            'results': input_results,
            'target': np.array([target_dx, target_dy], dtype=np.float32),
            'last_abs_coords': np.array([last_x, last_y], dtype=np.float32),
            'episode_id': episode_id,
            'meta_features': meta
        })

    return sequences

print("✅ build_train_sequences 정의 완료 (점수차 포함)")

✅ build_train_sequences 정의 완료 (점수차 포함)


In [ ]:
# 시퀀스 구축 (원본 데이터로 - 증강은 Train/Val 분할 후 적용)
print("📦 시퀀스 구축 중...")
sequences = build_train_sequences(
    train_df, type_map, result_map, match_context,
    role_map, player_stats_map, team_stats_map  # ← team_stats_map 추가
)
print(f"✅ 총 {len(sequences):,}개 시퀀스 (원본)")

# 메타 피처 개수 확인
sample_meta = sequences[0]['meta_features']
print(f"✅ Meta Features: {len(sample_meta)}개")

📦 시퀀스 구축 중...


Train 시퀀스 구축: 100%|██████████| 15435/15435 [00:50<00:00, 308.57it/s]

✅ 총 15,428개 시퀀스 (원본)
✅ Meta Features: 48개


---
## 07. LSTM 모델

In [ ]:
class LSTMEncoder(nn.Module):
    """LSTM Encoder with Simple Attention (v8.9 - MHA 제거)"""

    def __init__(self, num_types, num_results, embedding_dim=16, hidden_dim=128,
                 num_layers=2, dropout=0.3, continuous_dim=9):
        super().__init__()
        self.hidden_dim = hidden_dim

        # Embeddings
        self.type_emb = nn.Embedding(num_types, embedding_dim)
        self.result_emb = nn.Embedding(num_results, embedding_dim)

        # LSTM
        input_dim = continuous_dim + embedding_dim * 2
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )

        # Simple Attention
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )

        # Output head
        self.head = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, continuous, types, results, lengths, return_embedding=False):
        batch_size = continuous.size(0)
        max_len = continuous.size(1)
        device = continuous.device

        # Embedding concat
        x = torch.cat([
            continuous,
            self.type_emb(types),
            self.result_emb(results)
        ], dim=-1)

        # LSTM
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        lstm_out, _ = self.lstm(packed)
        lstm_out, _ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

        # Padding mask
        mask = torch.arange(max_len, device=device).expand(batch_size, max_len) >= lengths.to(device).unsqueeze(1)

        # Simple Attention
        attn_weights = self.attention(lstm_out).squeeze(-1)
        attn_weights = attn_weights.masked_fill(mask, float('-inf'))
        attn_weights = torch.softmax(attn_weights, dim=1)
        context = torch.bmm(attn_weights.unsqueeze(1), lstm_out).squeeze(1)

        pred = self.head(context)

        if return_embedding:
            return pred, context
        return pred

    def extract_embedding(self, continuous, types, results, lengths):
        with torch.no_grad():
            _, emb = self.forward(continuous, types, results, lengths, return_embedding=True)
        return emb


print("✅ LSTM + Simple Attention 모델 정의 완료 (v8.9 - MHA 제거)")

✅ LSTM + Simple Attention 모델 정의 완료 (v8.9 - MHA 제거)


---
## 08. Dataset & DataLoader

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, sequences, is_train=True):
        self.sequences = sequences
        self.is_train = is_train

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        item = {
            'continuous': torch.tensor(seq['continuous'], dtype=torch.float32),
            'types': torch.tensor(seq['types'], dtype=torch.long),
            'results': torch.tensor(seq['results'], dtype=torch.long),
            'last_abs_coords': torch.tensor(seq['last_abs_coords'], dtype=torch.float32),
            'meta_features': seq['meta_features']
        }
        if self.is_train:
            item['target'] = torch.tensor(seq['target'], dtype=torch.float32)
        return item


def collate_fn(batch):
    continuous = pad_sequence([b['continuous'] for b in batch], batch_first=True)
    types = pad_sequence([b['types'] for b in batch], batch_first=True)
    results = pad_sequence([b['results'] for b in batch], batch_first=True)
    last_abs = torch.stack([b['last_abs_coords'] for b in batch])
    lengths = torch.tensor([len(b['continuous']) for b in batch])

    # Meta features를 리스트로
    meta_list = [b['meta_features'] for b in batch]

    result = {
        'continuous': continuous,
        'types': types,
        'results': results,
        'lengths': lengths,
        'last_abs_coords': last_abs,
        'meta_features': meta_list
    }

    if 'target' in batch[0]:
        result['target'] = torch.stack([b['target'] for b in batch])

    return result

print("✅ Dataset 정의 완료")

✅ Dataset 정의 완료


---
## 09. LSTM 학습

In [ ]:
# Train/Val 분할 (증강 전!)
train_seqs_raw, val_seqs = train_test_split(sequences, test_size=0.15, random_state=SEED)
print(f"분할 후 - Train: {len(train_seqs_raw):,}, Val: {len(val_seqs):,}")

# Train만 증강 (Val은 원본 유지 → 데이터 누수 방지)
train_seqs = augment_y_flip_sequences(train_seqs_raw)
print(f"증강 후 - Train: {len(train_seqs):,} (원본의 2배), Val: {len(val_seqs):,} (원본 유지)")

train_loader = DataLoader(
    SequenceDataset(train_seqs, is_train=True),
    batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
)
val_loader = DataLoader(
    SequenceDataset(val_seqs, is_train=True),
    batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn
)

print(f"✅ DataLoader 생성 완료")

분할 후 - Train: 13,113, Val: 2,315
증강 후 - Train: 26,226 (원본의 2배), Val: 2,315 (원본 유지)
✅ DataLoader 생성 완료


In [ ]:
# LSTM 모델 생성
lstm_model = LSTMEncoder(
    NUM_TYPES, NUM_RESULTS,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=LSTM_HIDDEN_DIM,
    num_layers=LSTM_NUM_LAYERS,
    dropout=LSTM_DROPOUT,
    continuous_dim=LSTM_CONTINUOUS_DIM
).to(DEVICE)

optimizer = torch.optim.Adam(lstm_model.parameters(), lr=LR, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

print(f"LSTM Parameters: {sum(p.numel() for p in lstm_model.parameters()):,}")

LSTM Parameters: 638,131


In [ ]:
print("\n🚀 LSTM 학습 시작...")
best_val_dist = float('inf')
best_state = None

for epoch in range(1, LSTM_EPOCHS + 1):
    # Train
    lstm_model.train()
    train_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        pred = lstm_model(
            batch['continuous'].to(DEVICE),
            batch['types'].to(DEVICE),
            batch['results'].to(DEVICE),
            batch['lengths']
        )
        loss = F.huber_loss(pred, batch['target'].to(DEVICE), delta=0.15)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    lstm_model.eval()
    val_dists = []
    with torch.no_grad():
        for batch in val_loader:
            pred = lstm_model(
                batch['continuous'].to(DEVICE),
                batch['types'].to(DEVICE),
                batch['results'].to(DEVICE),
                batch['lengths']
            )
            pred = pred.cpu().numpy()
            last_abs = batch['last_abs_coords'].numpy()
            target = batch['target'].numpy()

            pred_abs = (pred + last_abs) * [105, 68]
            true_abs = (target + last_abs) * [105, 68]
            dists = np.sqrt(np.sum((pred_abs - true_abs)**2, axis=1))
            val_dists.extend(dists)

    mean_dist = np.mean(val_dists)
    scheduler.step(mean_dist)

    if mean_dist < best_val_dist:
        best_val_dist = mean_dist
        best_state = {k: v.cpu().clone() for k, v in lstm_model.state_dict().items()}

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Val Distance = {mean_dist:.2f}m (best: {best_val_dist:.2f}m)")

# Best 모델 로드
lstm_model.load_state_dict(best_state)
print(f"\n✅ LSTM 학습 완료: Best = {best_val_dist:.2f}m")


🚀 LSTM 학습 시작...
Epoch 10: Val Distance = 15.54m (best: 15.50m)
Epoch 20: Val Distance = 15.33m (best: 15.20m)
Epoch 30: Val Distance = 15.16m (best: 15.08m)
Epoch 40: Val Distance = 14.97m (best: 14.97m)
Epoch 50: Val Distance = 14.93m (best: 14.93m)
Epoch 60: Val Distance = 14.93m (best: 14.90m)
Epoch 70: Val Distance = 14.89m (best: 14.87m)
Epoch 80: Val Distance = 14.94m (best: 14.87m)
Epoch 90: Val Distance = 14.92m (best: 14.87m)
Epoch 100: Val Distance = 14.92m (best: 14.87m)

✅ LSTM 학습 완료: Best = 14.87m


---
## 10. LGBM 피처 추출 및 학습

In [ ]:
def extract_features_for_lgbm(sequences, lstm_model, device, is_train=True, batch_size=256):
    """LSTM 임베딩 + 메타 피처 결합"""
    lstm_model.eval()

    embeddings = []
    meta_feats = []
    targets = []
    last_abs_list = []

    loader = DataLoader(
        SequenceDataset(sequences, is_train=is_train),
        batch_size=batch_size, shuffle=False, collate_fn=collate_fn
    )

    # 메타 피처 키 정렬 (일관성)
    meta_keys = sorted(sequences[0]['meta_features'].keys())

    with torch.no_grad():
        for batch in tqdm(loader, desc="Feature Extraction"):
            # LSTM 임베딩 추출
            emb = lstm_model.extract_embedding(
                batch['continuous'].to(device),
                batch['types'].to(device),
                batch['results'].to(device),
                batch['lengths']
            )
            embeddings.append(emb.cpu().numpy())
            last_abs_list.append(batch['last_abs_coords'].numpy())

            # 메타 피처
            for meta in batch['meta_features']:
                meta_feats.append([meta.get(k, 0) for k in meta_keys])

            if is_train:
                targets.append(batch['target'].numpy())

    # 결합
    X_emb = np.vstack(embeddings)
    X_meta = np.array(meta_feats)
    X = np.hstack([X_emb, X_meta])
    last_coords = np.vstack(last_abs_list)

    feature_names = [f'lstm_emb_{i}' for i in range(X_emb.shape[1])] + meta_keys

    if is_train:
        return X, np.vstack(targets), last_coords, feature_names
    return X, last_coords, feature_names

print("✅ LGBM 피처 추출 함수 정의 완료")

✅ LGBM 피처 추출 함수 정의 완료


In [ ]:
# 피처 추출
print("\n📊 LGBM 피처 추출...")
X_train, y_train, last_abs_train, feature_names = extract_features_for_lgbm(
    train_seqs, lstm_model, DEVICE, is_train=True
)
X_val, y_val, last_abs_val, _ = extract_features_for_lgbm(
    val_seqs, lstm_model, DEVICE, is_train=True
)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"Features: {len(feature_names)}개 (LSTM: {LSTM_HIDDEN_DIM*2}, Meta: {len(feature_names) - LSTM_HIDDEN_DIM*2})")


📊 LGBM 피처 추출...


Feature Extraction: 100%|██████████| 10/10 [00:00<00:00, 37.15it/s]

X_train shape: (26226, 304)
X_val shape: (2315, 304)
Features: 304개 (LSTM: 256, Meta: 48)


In [ ]:
# LGBM 학습
print("\n🌲 LightGBM 학습...")

lgbm_x = lgb.LGBMRegressor(**LGBM_PARAMS)
lgbm_y = lgb.LGBMRegressor(**LGBM_PARAMS)

lgbm_x.fit(
    X_train, y_train[:, 0],
    eval_set=[(X_val, y_val[:, 0])],
    callbacks=[lgb.early_stopping(100, verbose=False)]
)

lgbm_y.fit(
    X_train, y_train[:, 1],
    eval_set=[(X_val, y_val[:, 1])],
    callbacks=[lgb.early_stopping(100, verbose=False)]
)

print(f"✅ LGBM X: {lgbm_x.best_iteration_} iterations")
print(f"✅ LGBM Y: {lgbm_y.best_iteration_} iterations")


🌲 LightGBM 학습...
✅ LGBM X: 231 iterations
✅ LGBM Y: 176 iterations


---
## 11. Validation 평가

In [ ]:
# 예측
pred_x = lgbm_x.predict(X_val)
pred_y = lgbm_y.predict(X_val)

# 절대좌표 복원
pred_abs_x = np.clip((pred_x + last_abs_val[:, 0]) * 105.0, 0, 105)
pred_abs_y = np.clip((pred_y + last_abs_val[:, 1]) * 68.0, 0, 68)

true_abs_x = (y_val[:, 0] + last_abs_val[:, 0]) * 105.0
true_abs_y = (y_val[:, 1] + last_abs_val[:, 1]) * 68.0

# 거리 계산
distances = np.sqrt((pred_abs_x - true_abs_x)**2 + (pred_abs_y - true_abs_y)**2)

print("="*50)
print(f"📊 Validation Results (Hybrid LSTM + LGBM)")
print("="*50)
print(f"Mean Distance Error: {np.mean(distances):.2f}m")
print(f"Median Distance Error: {np.median(distances):.2f}m")
print(f"Std Distance Error: {np.std(distances):.2f}m")
print("="*50)

📊 Validation Results (Hybrid LSTM + LGBM)
Mean Distance Error: 14.47m
Median Distance Error: 11.91m
Std Distance Error: 10.86m


---
## 12. Feature Importance 분석

In [ ]:
import matplotlib.pyplot as plt

# Feature Importance
importance_x = lgbm_x.feature_importances_
importance_y = lgbm_y.feature_importances_
avg_importance = (importance_x + importance_y) / 2

# Top 30
top_idx = np.argsort(avg_importance)[-30:][::-1]
top_features = [feature_names[i] for i in top_idx]
top_importance = avg_importance[top_idx]

plt.figure(figsize=(12, 10))
plt.barh(range(len(top_features)), top_importance[::-1])
plt.yticks(range(len(top_features)), top_features[::-1])
plt.xlabel('Importance')
plt.title('Top 30 Feature Importance (LSTM Embedding + Meta Features)')
plt.tight_layout()
plt.show()

# LSTM vs Meta 비중
lstm_imp = np.sum(avg_importance[:LSTM_HIDDEN_DIM*2])
meta_imp = np.sum(avg_importance[LSTM_HIDDEN_DIM*2:])
print(f"\n📊 Feature Group Importance:")
print(f"   LSTM Embedding: {lstm_imp:.1f} ({lstm_imp/(lstm_imp+meta_imp)*100:.1f}%)")
print(f"   Meta Features:  {meta_imp:.1f} ({meta_imp/(lstm_imp+meta_imp)*100:.1f}%)")

---
## 13. 모델 저장

In [ ]:
import pickle

SAVE_DIR = f"{BASE_DIR}/models"
os.makedirs(SAVE_DIR, exist_ok=True)

# LSTM 저장
torch.save({
    'model_state_dict': lstm_model.state_dict(),
    'num_types': NUM_TYPES,
    'num_results': NUM_RESULTS,
    'hidden_dim': LSTM_HIDDEN_DIM,
    'embedding_dim': EMBEDDING_DIM,
}, f"{SAVE_DIR}/lstm_encoder.pt")
print("✅ LSTM 저장")

# LGBM 저장
lgbm_x.booster_.save_model(f"{SAVE_DIR}/lgbm_x.txt")
lgbm_y.booster_.save_model(f"{SAVE_DIR}/lgbm_y.txt")
print("✅ LGBM 저장")

# 매핑 저장
with open(f"{SAVE_DIR}/mappings.pkl", 'wb') as f:
    pickle.dump({
        'type_map': type_map,
        'result_map': result_map,
        'role_map': role_map,
        'player_stats_map': player_stats_map,
        'feature_names': feature_names
    }, f)
print("✅ 매핑 저장 완료")

##14. EDA

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import numpy as np
import pandas as pd

# 한글 폰트 설정 (Colab)
!apt-get install -y fonts-nanum > /dev/null 2>&1
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
if os.path.exists(font_path):
    font_name = fm.FontProperties(fname=font_path).get_name()
    fm.fontManager.addfont(font_path)
    plt.rcParams['font.family'] = font_name
plt.rcParams['axes.unicode_minus'] = False

print("✅ 시각화 설정 완료")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 히스토그램
axes[0].hist(distances, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(np.mean(distances), color='red', linestyle='--', label=f'Mean: {np.mean(distances):.2f}m')
axes[0].axvline(np.median(distances), color='orange', linestyle='--', label=f'Median: {np.median(distances):.2f}m')
axes[0].set_xlabel('Distance Error (m)')
axes[0].set_ylabel('Count')
axes[0].set_title('Error Distribution')
axes[0].legend()

# Box plot
axes[1].boxplot(distances, vert=True)
axes[1].set_ylabel('Distance Error (m)')
axes[1].set_title('Error Box Plot')

# CDF
sorted_dist = np.sort(distances)
cdf = np.arange(1, len(sorted_dist) + 1) / len(sorted_dist)
axes[2].plot(sorted_dist, cdf, color='steelblue')
axes[2].axhline(0.5, color='gray', linestyle='--', alpha=0.5)
axes[2].axhline(0.9, color='gray', linestyle='--', alpha=0.5)
axes[2].set_xlabel('Distance Error (m)')
axes[2].set_ylabel('Cumulative Probability')
axes[2].set_title('Cumulative Distribution')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 통계 출력
print("\n📊 오차 통계:")
print(f"  Mean:   {np.mean(distances):.2f}m")
print(f"  Median: {np.median(distances):.2f}m")
print(f"  Std:    {np.std(distances):.2f}m")
print(f"  Min:    {np.min(distances):.2f}m")
print(f"  Max:    {np.max(distances):.2f}m")
print(f"  Q25:    {np.percentile(distances, 25):.2f}m")
print(f"  Q75:    {np.percentile(distances, 75):.2f}m")
print(f"  Q90:    {np.percentile(distances, 90):.2f}m")

##Zone 별 오차 분석

In [ ]:
# Zone별 오차 계산
zone_errors = {i: [] for i in range(9)}
zone_names = ['DEF_LEFT', 'DEF_CENTER', 'DEF_RIGHT',
              'MID_LEFT', 'MID_CENTER', 'MID_RIGHT',
              'ATT_LEFT', 'ATT_CENTER', 'ATT_RIGHT']

for i, seq in enumerate(val_seqs):
    zone_id = seq['meta_features'].get('zone_id', 4)
    zone_id = min(max(int(zone_id), 0), 8)
    zone_errors[zone_id].append(distances[i])

# Zone별 통계
zone_stats = []
for z in range(9):
    if zone_errors[z]:
        zone_stats.append({
            'zone': zone_names[z],
            'count': len(zone_errors[z]),
            'mean': np.mean(zone_errors[z]),
            'median': np.median(zone_errors[z]),
            'std': np.std(zone_errors[z])
        })

zone_df = pd.DataFrame(zone_stats)
print("📊 Zone별 오차:")
print(zone_df.to_string(index=False))

In [ ]:
# Zone 히트맵
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Mean Error 히트맵
mean_grid = np.zeros((3, 3))
count_grid = np.zeros((3, 3))
for z in range(9):
    row, col = z // 3, z % 3
    if zone_errors[z]:
        mean_grid[row, col] = np.mean(zone_errors[z])
        count_grid[row, col] = len(zone_errors[z])

im1 = axes[0].imshow(mean_grid, cmap='RdYlGn_r', aspect='auto')
axes[0].set_xticks([0, 1, 2])
axes[0].set_yticks([0, 1, 2])
axes[0].set_xticklabels(['Left', 'Center', 'Right'])
axes[0].set_yticklabels(['Defense', 'Midfield', 'Attack'])
axes[0].set_title('Mean Distance Error by Zone (m)')
for i in range(3):
    for j in range(3):
        axes[0].text(j, i, f'{mean_grid[i, j]:.1f}', ha='center', va='center', fontsize=14, fontweight='bold')
plt.colorbar(im1, ax=axes[0])

# Sample Count 히트맵
im2 = axes[1].imshow(count_grid, cmap='Blues', aspect='auto')
axes[1].set_xticks([0, 1, 2])
axes[1].set_yticks([0, 1, 2])
axes[1].set_xticklabels(['Left', 'Center', 'Right'])
axes[1].set_yticklabels(['Defense', 'Midfield', 'Attack'])
axes[1].set_title('Sample Count by Zone')
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, f'{int(count_grid[i, j])}', ha='center', va='center', fontsize=14, fontweight='bold')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance
importance_x = lgbm_x.feature_importances_
importance_y = lgbm_y.feature_importances_
avg_importance = (importance_x + importance_y) / 2

# LSTM vs Meta 분리
lstm_dim = LSTM_HIDDEN_DIM * 2  # Bidirectional
lstm_importance = avg_importance[:lstm_dim]
meta_importance = avg_importance[lstm_dim:]
meta_names = feature_names[lstm_dim:]

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Meta Features Top 25
top_meta_idx = np.argsort(meta_importance)[-25:][::-1]
top_meta_names = [meta_names[i] for i in top_meta_idx]
top_meta_values = meta_importance[top_meta_idx]

axes[0].barh(range(len(top_meta_names)), top_meta_values[::-1], color='coral')
axes[0].set_yticks(range(len(top_meta_names)))
axes[0].set_yticklabels(top_meta_names[::-1])
axes[0].set_xlabel('Importance')
axes[0].set_title('Top 25 Meta Features')

# LSTM Embedding Importance 분포
axes[1].hist(lstm_importance, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[1].axvline(np.mean(lstm_importance), color='red', linestyle='--',
                label=f'Mean: {np.mean(lstm_importance):.1f}')
axes[1].set_xlabel('Importance')
axes[1].set_ylabel('Count')
axes[1].set_title(f'LSTM Embedding Importance Distribution ({lstm_dim}D)')
axes[1].legend()

plt.tight_layout()
plt.show()

# 비중 출력
lstm_total = np.sum(lstm_importance)
meta_total = np.sum(meta_importance)
print(f"\n📊 Feature Group Contribution:")
print(f"  LSTM Embedding ({lstm_dim}D): {lstm_total:.1f} ({lstm_total/(lstm_total+meta_total)*100:.1f}%)")
print(f"  Meta Features ({len(meta_names)}D):  {meta_total:.1f} ({meta_total/(lstm_total+meta_total)*100:.1f}%)")

In [ ]:
# 예측값 계산 (이미 위에서 했다면 생략)
pred_x = lgbm_x.predict(X_val)
pred_y = lgbm_y.predict(X_val)
pred_abs_x = np.clip((pred_x + last_abs_val[:, 0]) * 105.0, 0, 105)
pred_abs_y = np.clip((pred_y + last_abs_val[:, 1]) * 68.0, 0, 68)
true_abs_x = (y_val[:, 0] + last_abs_val[:, 0]) * 105.0
true_abs_y = (y_val[:, 1] + last_abs_val[:, 1]) * 68.0

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# X 좌표
axes[0].scatter(true_abs_x, pred_abs_x, alpha=0.2, s=5)
axes[0].plot([0, 105], [0, 105], 'r--', label='Perfect')
axes[0].set_xlabel('True X (m)')
axes[0].set_ylabel('Predicted X (m)')
axes[0].set_title('X Coordinate: Predicted vs True')
axes[0].legend()
axes[0].set_xlim(0, 105)
axes[0].set_ylim(0, 105)

# Y 좌표
axes[1].scatter(true_abs_y, pred_abs_y, alpha=0.2, s=5)
axes[1].plot([0, 68], [0, 68], 'r--', label='Perfect')
axes[1].set_xlabel('True Y (m)')
axes[1].set_ylabel('Predicted Y (m)')
axes[1].set_title('Y Coordinate: Predicted vs True')
axes[1].legend()
axes[1].set_xlim(0, 68)
axes[1].set_ylim(0, 68)

# 필드 위 분포 (샘플링)
sample_idx = np.random.choice(len(pred_abs_x), min(500, len(pred_abs_x)), replace=False)
axes[2].scatter(true_abs_x[sample_idx], true_abs_y[sample_idx],
                alpha=0.5, s=20, c='blue', label='True', marker='o')
axes[2].scatter(pred_abs_x[sample_idx], pred_abs_y[sample_idx],
                alpha=0.5, s=20, c='red', label='Predicted', marker='x')
# 필드 라인
axes[2].axvline(52.5, color='gray', linestyle='-', alpha=0.3)
axes[2].axhline(34, color='gray', linestyle='-', alpha=0.3)
axes[2].set_xlim(0, 105)
axes[2].set_ylim(0, 68)
axes[2].set_xlabel('X (m)')
axes[2].set_ylabel('Y (m)')
axes[2].set_title('Field Distribution (500 samples)')
axes[2].legend()
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

# Part 2: Test 예측 및 제출 파일 생성

1.   항목 추가
2.   항목 추가



In [ ]:
def build_test_sequences(test_meta, base_dir, type_map, result_map, match_context,
                         role_map, player_stats_map, team_stats_map):
    """Test 시퀀스 구축 (v9.2: 점수차 포함)"""
    sequences = []
    episode_ids = []
    mc_dict = match_context.set_index('game_id').to_dict('index')

    for _, row in tqdm(test_meta.iterrows(), total=len(test_meta), desc="Test 시퀀스 구축"):
        episode_id = row['game_episode']
        game_id = row['game_id']

        episode_df = pd.read_csv(os.path.join(base_dir, row['path']))
        episode_df = episode_df.sort_values('time_seconds').reset_index(drop=True)

        if pd.isna(episode_df.iloc[-1]['end_x']):
            input_df = episode_df.iloc[:-1].copy()
        else:
            input_df = episode_df.copy()

        if len(input_df) < 1:
            last_x = episode_df.iloc[0]['start_x'] / 105.0
            last_y = episode_df.iloc[0]['start_y'] / 68.0
            sequences.append({
                'continuous': np.zeros((1, 10), dtype=np.float32),  # 10D
                'types': np.array([0]),
                'results': np.array([0]),
                'last_abs_coords': np.array([last_x, last_y], dtype=np.float32),
                'meta_features': {k: 0 for k in sorted(LGBM_FEATURES)}
            })
            episode_ids.append(episode_id)
            continue

        sx = input_df['start_x'].values / 105.0
        sy = input_df['start_y'].values / 68.0
        ex = input_df['end_x'].values / 105.0
        ey = input_df['end_y'].values / 68.0

        max_time = input_df['time_seconds'].max()
        norm_times = input_df['time_seconds'].values / max_time if max_time > 0 else np.zeros(len(input_df))

        curr_types = input_df['type_name'].map(type_map).fillna(0).values.astype(int)
        curr_results = input_df['result_name'].fillna("Unknown").map(result_map).fillna(0).values.astype(int)

        # 팀 정보 & 점수차 계산
        team_id = input_df['team_id'].iloc[-1]
        score_diffs = calculate_score_diff(input_df, team_id, type_map)

        last_x, last_y = ex[-1], ey[-1]

        abs_coords = np.column_stack([ex, ey, norm_times])
        continuous = convert_to_relative(abs_coords, score_diffs)

        mc = mc_dict.get(game_id, {})
        player_id = input_df['player_id'].iloc[-1]
        role = role_map.get(player_id, 0)
        p_stats = player_stats_map.get(player_id, {'player_avg_dist': 0, 'player_avg_dx': 0})
        t_stats = team_stats_map.get(team_id, {'team_avg_pass_dist': 0, 'team_possession_ratio': 0})

        is_home = int(team_id == mc.get('home_team_id', -1))
        match_phase = int(input_df['time_seconds'].iloc[-1] // 900)
        cumulative_score_diff = score_diffs[-1] if len(score_diffs) > 0 else 0

        meta = compute_lgbm_features(
            ex, ey, sx, sy, norm_times, curr_types, curr_results,
            last_x, last_y, type_map,
            role=role,
            player_avg_dist=p_stats.get('player_avg_dist', 0),
            player_avg_dx=p_stats.get('player_avg_dx', 0),
            team_avg_pass_dist=t_stats.get('team_avg_pass_dist', 0),
            team_possession_ratio=t_stats.get('team_possession_ratio', 0),
            is_home=is_home,
            match_phase=match_phase,
            match_hour=mc.get('match_hour', 19),
            time_slot=mc.get('time_slot', 1),
            current_team_rest=mc.get('home_rest', 7) if is_home else mc.get('away_rest', 7),
            opp_team_rest=mc.get('away_rest', 7) if is_home else mc.get('home_rest', 7),
            rest_diff=mc.get('rest_diff', 0) if is_home else -mc.get('rest_diff', 0),
            cumulative_score_diff=cumulative_score_diff
        )

        sequences.append({
            'continuous': continuous.astype(np.float32),
            'types': curr_types,
            'results': curr_results,
            'last_abs_coords': np.array([last_x, last_y], dtype=np.float32),
            'meta_features': meta
        })
        episode_ids.append(episode_id)

    return sequences, episode_ids

print("✅ build_test_sequences 정의 완료 (점수차 포함)")

In [ ]:
# Test 데이터 로드 및 시퀀스 구축
print("📦 Test 시퀀스 구축 중...")
test_meta = pd.read_csv(TEST_PATH)
print(f"Test Episodes: {len(test_meta)}")

test_seqs, test_ids = build_test_sequences(
    test_meta, BASE_DIR, type_map, result_map, match_context,
    role_map, player_stats_map, team_stats_map  # ← team_stats_map 추가
)
print(f"✅ Test 시퀀스: {len(test_seqs)}개")

In [ ]:
# Test 피처 추출
print("\n📊 Test 피처 추출...")
X_test, last_abs_test, _ = extract_features_for_lgbm(
    test_seqs, lstm_model, DEVICE, is_train=False
)
print(f"X_test shape: {X_test.shape}")

In [ ]:
# 예측
print("\n🔮 Test 예측...")
test_pred_x = lgbm_x.predict(X_test)
test_pred_y = lgbm_y.predict(X_test)

# 절대좌표 복원
test_abs_x = np.clip((test_pred_x + last_abs_test[:, 0]) * 105.0, 0, 105)
test_abs_y = np.clip((test_pred_y + last_abs_test[:, 1]) * 68.0, 0, 68)

print(f"\n📊 예측값 통계:")
print(f"  end_x: mean={np.mean(test_abs_x):.2f}, std={np.std(test_abs_x):.2f}, range=[{np.min(test_abs_x):.1f}, {np.max(test_abs_x):.1f}]")
print(f"  end_y: mean={np.mean(test_abs_y):.2f}, std={np.std(test_abs_y):.2f}, range=[{np.min(test_abs_y):.1f}, {np.max(test_abs_y):.1f}]")

# ========== 제출 파일 생성 ==========
print("\n📁 제출 파일 생성...")

submission_df = pd.DataFrame({
    'game_episode': test_ids,
    'end_x': test_abs_x,
    'end_y': test_abs_y
})

SUBMIT_PATH = f"{BASE_DIR}/submission_v9_2_8_Hubor.csv"
submission_df.to_csv(SUBMIT_PATH, index=False)

print(f"✅ 제출 파일 저장: {SUBMIT_PATH}")
print(f"   총 {len(submission_df)}개 예측")

print("\n📋 제출 파일 샘플 (처음 5개):")
print(submission_df.head().to_string(index=False))